# Setup

 - [OpenAI](https://beta.openai.com/)
 - [python-dotenv](https://pypi.org/project/python-dotenv/)

        python -m venv venv
        source venv/bin/activate
        pip install openai python-dotenv
        pip freeze >> requirements.txt

 - **.env**

     - [OpenAI Key](https://beta.openai.com/account/api-keys)

            OPENAI_API_KEY=

In [ ]:
from subprocess import run


run(["pip", "install", "openai", "python-dotenv"])

In [ ]:
from os import path


def all_required_variables_have_been_provided(config, variables):
    ok = True
    for variable in variables:
        if variable not in config or not config[variable]:
            ok = False
            print(f'{variable} is required')
    return ok


def create_environment_file(**kwargs):
    if not all_required_variables_have_been_provided(kwargs, [
        'OPENAI_API_KEY'
    ]):
        return

    with open(path.join('.', '.env'), 'w') as env_file:
        for key, value in kwargs.items():
            env_file.write(f'{key}={value}')


create_environment_file(OPENAI_API_KEY='')

In [ ]:
from dotenv import load_dotenv
from os import getenv

load_dotenv()

print(getenv('OPENAI_API_KEY'))

# [Models](https://beta.openai.com/docs/models)

 - [GPT-3](https://beta.openai.com/docs/models/gpt-3): understand and generate natural language.
     - text-davinci-003
     - text-curie-001
     - text-babbage-001
     - text-ada-001
 - [Codex](https://beta.openai.com/docs/models/codex): understand and generate code, including translating natural language to code.
     - code-davinci-002
     - code-cushman-001
 - [Moderation](https://beta.openai.com/docs/guides/moderation): can detect whether text may be sensitive or unsafe

In [115]:
import openai

openai.api_key = getenv('OPENAI_API_KEY')

def completion_config(model, prompt, **kwargs):
    config = {
        'model': model,
        'prompt': prompt
    }

    return {**config, **kwargs}


def get_text_from_response(response):
        return response['choices'][0]['text']

# [Text classification](https://monkeylearn.com/text-classification/)

Is a machine learning technique that assigns a set of predefined
categories to open-ended text. Text classifiers can be used to organize, structure, and categorize documents, medical studies 
and files.

Text classification is one of the fundamental tasks in natural 
language processing with broad applications such as sentiment 
analysis, topic labeling, spam detection, and intent detection.

In [ ]:
def perform_text_classification(prompt, **kwargs):
    config = completion_config(
        'text-davinci-003',
        prompt,
        **kwargs
    )

    return openai.Completion.create(**config)


def build_prompt_for_this_tweet(tweet):
    return f"""Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: "{tweet}"
Sentiment:"""


def build_prompt_for_this_tweets(tweets):
    parsed_tweets = ''

    for index, tweet in enumerate(tweets):
        parsed_tweets += f'{index + 1}. "{tweet}"\n'

    return f"""Classify the sentiment in these tweets:

{parsed_tweets}

Tweet sentiment ratings:"""

In [ ]:
response = perform_text_classification(build_prompt_for_this_tweet(
    "I loved the new Batman movie!"
),
    temperature = 0,
    max_tokens = 60,
    top_p = 1,
    frequency_penalty = 0.5,
    presence_penalty = 0
)

print(response)

In [ ]:
response = perform_text_classification(build_prompt_for_this_tweets([
    "I can't stand homework",
    "This sucks. I'm bored 😠",
    "I can't wait for Halloween!!!",
    "My cat is adorable ❤️❤️",
    "I hate chocolate"
]),
    temperature = 0,
    max_tokens = 60,
    top_p = 1,
    frequency_penalty = 0,
    presence_penalty = 0
)

print(response)

# [Text Generation](https://spotintelligence.com/2022/12/19/text-generation-nlp/)

Is a subfield of natural language processing (NLP) that deals 
with generating text automatically. It has a wide range of 
applications, including machine translation, content creation, 
and conversational agents.

In [89]:
def perform_text_generation(prompt, **kwargs):
    config = completion_config(
        'text-davinci-003',
        prompt,
        **{
            'temperature': 0.6,
            'max_tokens': 150,
            'top_p': 1,
            'frequency_penalty': 1,
            'presence_penalty': 1,
            **kwargs
        }
    )

    return openai.Completion.create(**config)

In [ ]:
response = perform_text_generation(
    "Brainstorm some ideas combining Golang and Network programming:",
)

print(response)

# [Conversational AI](https://www.ibm.com/topics/conversational-ai)

Conversational artificial intelligence refers to techonologies, 
like chatbots or virtual agents, which users can talk to. They 
use large volumes of data, machine learning, and natural 
language processing to help imitate human interactions, 
recognizing speech and text inputs and translating their meanings
across various languages.

In [98]:
def perform_conversation(prompt, **kwargs):
    config = completion_config(
        'text-davinci-003',
        prompt,
        **{
            'temperature': 0.9,
            'max_tokens': 150,
            'top_p': 1,
            'frequency_penalty': 0,
            'presence_penalty': 0.6,
            'stop': [' Human:', ' AI:'],
            **kwargs
        }
    )

    return openai.Completion.create(**config)


def run_jarvis(message):
    jarvis_config = f"""The following is a conversation with an AI assistant \
called Jarvis. The assistant is helpful, creative and clever.

Human: {message}
AI:"""

    return perform_conversation(jarvis_config)

In [ ]:
def jarvis_chat():
    print("To exit type 'Bye'")

    while (message := input('Human: ')) != 'Bye':
        response = run_jarvis(message.strip())
        print(f'Jarvis:{get_text_from_response(response)}')


jarvis_chat()

# Text Transformation

In [116]:
def perform_transformation(prompt, **kwargs):
    config = completion_config(
        'text-davinci-003',
        prompt,
        **kwargs
    )

    return openai.Completion.create(**config)

## Text Translation

In [121]:
def translate_the_text_to(text, list_of_languages):
    languages = ''
    for index, language in enumerate(list_of_languages):
        languages += f'{index + 1}. {language} '

    prompt = f"""Translate this into {languages.strip()}:

{text}

1."""

    response = perform_transformation(prompt, **{
        'temperature': 0.3,
        'max_tokens': 100,
        'top_p': 1,
        'frequency_penalty': 0,
        'presence_penalty': 0,
    })

    prompt += get_text_from_response(response)

    print(prompt)

In [ ]:
translate_the_text_to(
    'What rooms do you have available?',
    ['french', 'spanish', 'japanese']
)

## Text Conversion

In [117]:
def to_emoji(text):
    prompt = f"""Represent this phrase with emojis.

{text}:"""

    response = perform_transformation(prompt, **{
        'temperature': 0.8,
        'max_tokens': 60,
        'top_p': 1,
        'frequency_penalty': 0,
        'presence_penalty': 0,
    })

    prompt += get_text_from_response(response)

    print(prompt)

In [ ]:
to_emoji('Alan Turing, father of computer science')

## Summarization

In [123]:
def summarize_to(level, text):
    prompt = f"""Summarize this for a {level} student:

{text}"""

    response = perform_transformation(prompt, **{
        'temperature': 0.7,
        'max_tokens': 256,
        'top_p': 1,
        'frequency_penalty': 0,
        'presence_penalty': 0,
    })

    prompt += '\n\n' + get_text_from_response(response)

    print(prompt)

In [ ]:
summarize_to('second-grade',
    """Computability is the ability to solve a problem in an effective manner.
It is a key topic of the field of computability theory within mathematical
logic and the theory of computation within computer science. The computability
of a problem is closely linked to the existence of an algorithm to solve the problem.

The most widely studied models of computability are the Turing-computable and
μ-recursive functions, and the lambda calculus, all of which have computationally
equivalent power. Other forms of computability are studied as well: computability
notions weaker than Turing machines are studied in automata theory, while
computability notions stronger than Turing machines are studied in the field of
hypercomputation."""
)

## Text Completion

In [ ]:
def complete_this_text(text):
    response = openai.Completion.create(**{
        'model': 'text-davinci-001',
        'prompt': text,
        'temperature': 0.29,
        'max_tokens': 64,
        'top_p': 1,
        'frequency_penalty': 0,
        'presence_penalty': 0,
    })

    print(text + get_text_from_response(response))

In [ ]:
complete_this_text('Here all suspicion needs')

In [129]:
def complete_this_code(code):
    response = perform_transformation(code, **{
        'temperature': 0.7,
        'max_tokens': 64,
        'top_p': 1,
        'frequency_penalty': 0,
        'presence_penalty': 0,
    })

    print(code + get_text_from_response(response))

In [ ]:
complete_this_code(
    """struct Entry {
    void* key;
    void* value"""
)

## Factual responses

In [142]:
def make_this_question(question):
    prompt = f'Q: {question}\nA:'

    response = perform_transformation(prompt, **{
        'temperature': 0,
        'max_tokens': 64,
        'top_p': 1,
        'frequency_penalty': 0,
        'presence_penalty': 0,
        'stop': ['\n\n']
    })

    print(f'Q: {question}\n\nA:{get_text_from_response(response)}')

In [ ]:
make_this_question('Who is Donald Knuth?')